# Извлечение данных из коллекции новостных текстов

Данные расположены  [в файле data/news.tar.gz](data/news.tar.gz). С некоторых новостных сайтов был загружен архив новостей а  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`
    

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

        sport <tab> Сборная Канады по хоккею разгромила чехов <tab> Сборная Канады по хоккею крупно об...

С помощью [Yargy](https://github.com/natasha/yargy) или [Томита-парсера](https://github.com/yandex/tomita-parser) извлеките данные, которые можно описать структурой вида:

@dataclass

class Entry:

    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]


In [1]:
!pip install yargy


  Using cached yargy-0.16.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached pymorphy2-0.9.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl.metadata (7.0 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
Using cached yargy-0.16.0-py3-none-any.whl (33 kB)
Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)


In [23]:
from dataclasses import dataclass
from yargy.predicates import lte, gte, dictionary
from yargy.predicates import gram, is_capitalized, dictionary, normalized
from yargy.relations import gnc_relation
from yargy import Parser, rule, and_, or_
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact

In [31]:
@dataclass
class Entry:
    name: str
    birth_date: Optional[str] = None
    birth_place: Optional[str] = None

In [44]:
gnc = gnc_relation()
Person = fact(
    "Person",
    ["first", "last"]
)

Entry = fact(
    "Entry",
    ["name", "birth_date", "birth_place"]
)

#имя фамилия
NAME = rule(
    gram("Name"),
    gram("Surn")
)


MONTH_WORDS = dictionary({
    "января", "февраля", "марта", "апреля", "мая", "июня",
    "июля", "августа", "сентября", "октября", "ноября", "декабря"
})

DAY_NUM = and_(gte(1), lte(31))
MONTH_NUM = and_(gte(1), lte(12))
YEAR_NUM = and_(gte(1800), lte(2025))

#даты
DATE_RULE = or_(
    rule(YEAR_NUM, '.', MONTH_NUM, '.', DAY_NUM), # 2001.01.21
    rule(YEAR_NUM, '-', MONTH_NUM, '-', DAY_NUM), # 2001-01-21
    rule(DAY_NUM, '-', MONTH_NUM, '-', YEAR_NUM), # 21-01-2001
    rule(DAY_NUM, '.', MONTH_NUM, '.', YEAR_NUM), # 21.01.2001
    rule(DAY_NUM, MONTH_WORDS, YEAR_NUM),         # 21 января 2001
    rule(YEAR_NUM, 'году'),                       # 2001 году
    rule(YEAR_NUM, 'г', '.')                      # 2001 г.
).named('DATE')


BIRTH_WORDS = morph_pipeline([
    "появился на свет",
    "родился",
    "дата рождения",
    "был рожден",
    "рождение"
])

BIRTH_PLACE = rule(
    and_(
        gram("NOUN"),
        is_capitalized()
    ).optional().repeatable()
)

BIRTH_PLACE_TYPE = rule(
    gram("PREP"),
    dictionary({
        "городе",
        "хуторе",
        "селе",
        "поселке",
        "деревне",
        "мегаполисе"
    }).optional(),
)

SENT = rule(
    NAME.interpretation(Entry.name),
    or_(
        rule(
            BIRTH_WORDS,
            DATE.interpretation(Entry.birth_date),
            BIRTH_PLACE_TYPE,
            BIRTH_PLACE.interpretation(Entry.birth_place.normalized().custom(str.title))
        ),
        rule(
            BIRTH_WORDS,
            DATE.interpretation(Entry.birth_date),
        ),
        rule(
            BIRTH_WORDS,
            BIRTH_PLACE_TYPE,
            BIRTH_PLACE.interpretation(Entry.birth_place.normalized().custom(str.title))
        ),
    ).optional()
    
).interpretation(Entry)

In [45]:
text = "Дмитрий Смирнов родился 29 октября 1982 в селе Овсянникино"
parser = Parser(SENT)
for match in parser.findall(text):
    print(match.fact)

Entry(name='Дмитрий Смирнов', birth_date='29 октября 1982', birth_place='Овсянникино')


In [48]:
text = "Сергей Александров родился 12.12.1990 в городе Рязань"
parser = Parser(SENT)
for match in parser.findall(text):
    print(match.fact)

Entry(name='Сергей Александров', birth_date='12.12.1990', birth_place='Рязань')


In [62]:
text = "Светлана Орлова появилась на свет 7 октября 2000 в поселке Ушаковка"
parser = Parser(SENT)
for match in parser.findall(text):
    print(match.fact)

Entry(name='Светлана Орлова', birth_date='7 октября 2000', birth_place='Ушаковка')


In [65]:
def parse_news(input_file_path, output_file_path):
    with open(input_file_path, "r", encoding="utf-8") as input_file, open(output_file_path, "w", encoding="utf-8") as output_file:
        for line in input_file:

            category, headline, article_text = line.strip().split("\t", 2)
            
            parser = Parser(SENT)
            matches = parser.findall(article_text)
            
            for match in matches:
                output_file.write(f"{match.fact}\n")

parse_news("data/news.txt", "result.txt")
print("Выполнено")

Выполнено
